### Hyperparameter tuning with grid search

In [1]:
import torch
from core.util.save_model import save_parameters
from sklearn.model_selection import ParameterGrid
from core.models import GRU
from core.models.model_training import blocked_training
from core.util.hyperparameter_configuration import get_hyperparameter_configuration

Set ML model, loss function and hyperparameters that that will be tested.

In [2]:
gridsearch_params = {
    "learning_rate": [0.001, 0.003, 0.005],
    "batch_size": [32, 64, 128],
    "lookback": [36, 48, 96],
    "num_layers": [1, 2],
}

# Extract hyperparameters configuration that will not be tuned upon
(
    hidden_size,
    EPOCHS,
    horizon,
    loss_function,
    dropout_rate,
    folds,
    early_stopper,
) = get_hyperparameter_configuration()

# Specific to experiment
features = {}
experiment_name = "00_GRU_trefor_park"
model_used = GRU
model_input_size = 1

Use CUDA (GPU) if available.

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

Train a model with specified hyperparameters

In [4]:
def train_with_params(params: dict) -> tuple[float, model_used]:
    """Train model with the specified hyperparameters."""
    # Extract hyperparameters
    batch_size = params["batch_size"]
    learning_rate = params["learning_rate"]
    num_layers = params["num_layers"]
    lookback = params["lookback"]

    # Initialize model
    model = model_used(
        input_size=model_input_size,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout_rate=dropout_rate,
        horizon=horizon,
        lookback=lookback,
    ).to(device)

    _, val_loss, best_model = blocked_training(
        model=model,
        learning_rate=learning_rate,
        device=device,
        batch_size=batch_size,
        lookback=lookback,
        early_stopper=early_stopper,
        features=features,
    )

    return min(val_loss), best_model

Iterate over all hyperparameters and train a model for each combination.

In [5]:
best_loss = float("inf")
best_params = None
best_model = None

for params in ParameterGrid(gridsearch_params):
    early_stopper.reset()
    v_loss, model = train_with_params(params)
    print(params, v_loss)
    if v_loss < best_loss:
        best_loss = v_loss
        best_params = params
        best_model = model

save_parameters(experiment_name, best_params)
print("Best Hyperparameters:", best_params)
print("Best Validation Loss:", best_loss)

Iterating epochs:   0%|          | 0/200 [00:00<?, ?it/s]

{'batch_size': 32, 'learning_rate': 0.001, 'lookback': 36, 'num_layers': 1} 0.03950754623842785


Iterating epochs:   0%|          | 0/200 [00:00<?, ?it/s]

{'batch_size': 32, 'learning_rate': 0.001, 'lookback': 36, 'num_layers': 2} 0.03998099955655853


Iterating epochs:   0%|          | 0/200 [00:00<?, ?it/s]

...

{'batch_size': 128, 'learning_rate': 0.005, 'lookback': 48, 'num_layers': 2} 0.5602075639313783


Iterating epochs:   0%|          | 0/200 [00:00<?, ?it/s]

{'batch_size': 128, 'learning_rate': 0.005, 'lookback': 96, 'num_layers': 1} 0.061035090450910316


Iterating epochs:   0%|          | 0/200 [00:00<?, ?it/s]

{'batch_size': 128, 'learning_rate': 0.005, 'lookback': 96, 'num_layers': 2} 0.13419393668336405
Best Hyperparameters: {'batch_size': 32, 'learning_rate': 0.001, 'lookback': 48, 'num_layers': 1}
Best Validation Loss: 0.03843020877372319
